# Automated Code Debugging Assistant (CRU-AI)

**Objective:**  
This notebook demonstrates an agentic system that can automatically:
1. Analyze Python code for syntax and logical errors
2. Correct the identified issues
3. Coordinate the workflow via a Manager agent

**Agents involved:**
- **Code Analyzer:** detects syntax/logical errors
- **Code Corrector:** fixes the identified issues
- **Manager:** oversees workflow execution

**Note:**  
We are not using OpenAI or Grok keys because they are either not available or not working as expected.  
Instead, we simulate the behavior of an LLM locally using Python functions and LiteLLM for demonstration purposes.  
This allows the notebook to run fully without any external API keys.


In [1]:
!pip install tokenizers==0.20.3
!pip install transformers accelerate sentencepiece --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing in

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

print("Libraries imported successfully")

Libraries imported successfully


In [3]:
#CELL 3 — Initialize Hugging Face LLM Pipeline
# Model: tiiuae/falcon-7b-instruct (free Hugging Face model)
model_name = "tiiuae/falcon-7b-instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",   # Use GPU if available
    torch_dtype=torch.float16
)

# Create Hugging Face text generation pipeline
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    do_sample=True,
    temperature=0.2
)

print("Hugging Face LLM initialized successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


Hugging Face LLM initialized successfully


In [4]:
#CELL 4 — Example Buggy Python Code
buggy_code = """
def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib
"""

print("Buggy Python code loaded")
print(buggy_code)


Buggy Python code loaded

def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib



In [5]:
#CELL 5 — Define Analyzer Agent
def analyze_code(code):
    prompt = f"""
You are a Python code analyzer.
Analyze the following Python code and list all syntax and logical errors clearly:

{code}
"""
    result = llm(prompt)[0]["generated_text"]
    return result

# Test Analyzer
analysis_report = analyze_code(buggy_code)
print("=== Analysis Report ===")
print(analysis_report)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


=== Analysis Report ===

You are a Python code analyzer.
Analyze the following Python code and list all syntax and logical errors clearly:


def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib

<p>Syntax errors:</p>

<ul>
<li>Incorrect syntax for the function name - <code>fibonacci</code> should be <code>fibonacci(n)</code></li>
<li>Incorrect syntax for the function arguments - <code>n</code> should be a number (not a string)</li>
<li>Incorrect syntax for the function body - <code>return []</code> should be <code>return fibonacci(n)</code></li>
</ul>

<p>Logical errors:</p>

<ul>
<li>The function is not returning anything, so the return statement is not necessary</li>
<li>The function is not taking any arguments, so the function name should be <code>fibonacci(n)</code></li>
</ul>

<p>The correcte

In [6]:
#CELL 6 — Define Corrector Agent
def correct_code(code, analysis):
    prompt = f"""
You are a Python code corrector.
Here is the code with its analysis report:
Code:
{code}

Analysis:
{analysis}

Provide the corrected version of the code, fully functional and properly formatted.
"""
    result = llm(prompt)[0]["generated_text"]
    return result

# Test Corrector
corrected_code = correct_code(buggy_code, analysis_report)
print("=== Corrected Code ===")
print(corrected_code)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


=== Corrected Code ===

You are a Python code corrector.
Here is the code with its analysis report:
Code:

def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib


Analysis:

You are a Python code analyzer.
Analyze the following Python code and list all syntax and logical errors clearly:


def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib

<p>Syntax errors:</p>

<ul>
<li>Incorrect syntax for the function name - <code>fibonacci</code> should be <code>fibonacci(n)</code></li>
<li>Incorrect syntax for the function arguments - <code>n</code> should be a number (not a string)</li>
<li>Incorrect syntax for the fun

In [7]:
#CELL 7 — Manager Agent (Workflow Orchestration)
def manager_workflow(code):
    print("Manager: Starting workflow...\n")

    # Step 1: Analyze
    analysis = analyze_code(code)
    print("Manager: Analysis complete.\n", analysis, "\n")

    # Step 2: Correct
    corrected = correct_code(code, analysis)
    print("Manager: Code correction complete.\n", corrected, "\n")

    print("Manager: Workflow executed successfully.")
    return corrected

# Run the complete CRU-AI workflow
final_code = manager_workflow(buggy_code)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Manager: Starting workflow...



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Manager: Analysis complete.
 
You are a Python code analyzer.
Analyze the following Python code and list all syntax and logical errors clearly:


def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib

<p>Syntax errors:</p>
<ul>
<li>The function name is not defined in the code.</li>
<li>The function is not defined in the code.</li>
<li>The function is defined twice in the code.</li>
</ul>
<p>Logical errors:</p>
<ul>
<li>The function is not defined in the code.</li>
<li>The function is defined twice in the code.</li>
</ul> 

Manager: Code correction complete.
 
You are a Python code corrector.
Here is the code with its analysis report:
Code:

def fibonacci(n):
    if n < 0:
        return []
    elif n == 1:
        return 0
    elif n == 1:
        return 0, 1
    else:
        fib = [0, 1]
        